In [49]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [51]:
import pandas as pd

In [52]:
#load in data (located on the first sheet)
df2 = pd.read_excel('LargeAssignmentProblemData.xlsx', sheet_name='Largeproblemdata',index_col=0)
df2.head()
times = df2.values.tolist()

In [53]:

#first index is the person, second index is the task, times[i][j] denotes the time it takes person i to complete task j

#declare a concrete model
model = ConcreteModel()

num_people=len(df2) #use i to index the people, this is the first index
num_tasks=len(df2.columns)  #use j to index the tasks, this is the second index

#declare the decision variables
model.x = Var(range(num_people), range(num_tasks), domain=NonNegativeReals) #NOTE: This is how you define 2-dimensional decision variables, model.x[i,j]

#set the objective
model.Objective = Objective(expr = sum(model.x[i,j]*times[i][j] for i in range(num_people) for j in range(num_tasks)), sense = minimize) #this is how you sum up over multiple indices

#set the people constraints
model.PeopleConstraints = ConstraintList() #declaring an empty list of constraints
for i in range(num_people): #add one constraint per person
    model.PeopleConstraints.add(expr = sum(model.x[i,j] for j in range(num_tasks)) <= 1)

#set the task constraints
model.TaskConstraints = ConstraintList()
for j in range(num_tasks): #adding one constraint per task
    model.TaskConstraints.add(expr = sum(model.x[i,j] for i in range(num_people)) >= 1)

#model.pprint()

In [54]:
#solve the model
opt = SolverFactory('cbc')
opt.solve(model, tee = False)

{'Problem': [{'Name': 'unknown', 'Lower bound': 2815.0, 'Upper bound': 2815.0, 'Number of objectives': 1, 'Number of constraints': 94, 'Number of variables': 2209, 'Number of nonzeros': 2209, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'System time': 0.03, 'Wallclock time': 0.03, 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': None, 'Number of created subproblems': None}, 'Black box': {'Number of iterations': 110}}, 'Error rc': 0, 'Time': 0.04663491249084473}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [55]:
#print solution
print("objective:", model.Objective())
for i in range(num_people):
    for j in range(num_tasks):
        if model.x[i,j]() > 0:
            print("person", i, "gets task", j)

objective: 2815.0
person 0 gets task 8
person 1 gets task 6
person 2 gets task 5
person 3 gets task 45
person 4 gets task 35
person 5 gets task 10
person 6 gets task 33
person 7 gets task 34
person 8 gets task 12
person 9 gets task 42
person 10 gets task 39
person 11 gets task 23
person 12 gets task 11
person 13 gets task 2
person 14 gets task 36
person 15 gets task 18
person 16 gets task 31
person 17 gets task 25
person 18 gets task 41
person 19 gets task 7
person 20 gets task 20
person 21 gets task 4
person 22 gets task 15
person 23 gets task 43
person 24 gets task 19
person 25 gets task 14
person 26 gets task 9
person 27 gets task 22
person 28 gets task 28
person 29 gets task 21
person 30 gets task 3
person 31 gets task 29
person 32 gets task 46
person 33 gets task 30
person 34 gets task 44
person 35 gets task 37
person 36 gets task 40
person 37 gets task 13
person 38 gets task 24
person 39 gets task 0
person 40 gets task 27
person 41 gets task 17
person 42 gets task 16
person 43 ge